In [38]:
import pandas as pd
import numpy as np
from datetime import datetime

In [39]:
pingviner = pd.read_csv('penguins.csv')
pingviner.dropna(inplace=True)
pingviner.reset_index(inplace=True)
pingviner.drop('index', axis=1, inplace=True)
pingviner.head(100)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
4,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE
...,...,...,...,...,...,...,...
95,Adelie,Biscoe,41.0,20.0,203.0,4725.0,MALE
96,Adelie,Biscoe,37.7,16.0,183.0,3075.0,FEMALE
97,Adelie,Biscoe,37.8,20.0,190.0,4250.0,MALE
98,Adelie,Biscoe,37.9,18.6,193.0,2925.0,FEMALE


In [40]:
# a)
vekt = pingviner.groupby(['sex'])['body_mass_g'].mean()
vekt

sex
FEMALE    3862.272727
MALE      4545.684524
Name: body_mass_g, dtype: float64

In [41]:
vekt2 = pingviner.groupby(['island', 'sex'])['body_mass_g'].mean()
vekt2

island     sex   
Biscoe     FEMALE    4319.375000
           MALE      5104.518072
Dream      FEMALE    3446.311475
           MALE      3987.096774
Torgersen  FEMALE    3395.833333
           MALE      4034.782609
Name: body_mass_g, dtype: float64

In [42]:
# c)
individer = pingviner.groupby(['species', 'island', 'sex'])['sex'].count()
individer

species    island     sex   
Adelie     Biscoe     FEMALE    22
                      MALE      22
           Dream      FEMALE    27
                      MALE      28
           Torgersen  FEMALE    24
                      MALE      23
Chinstrap  Dream      FEMALE    34
                      MALE      34
Gentoo     Biscoe     FEMALE    58
                      MALE      61
Name: sex, dtype: int64

In [43]:
# d)
adge = pingviner.groupby('species')['body_mass_g'].mean()
adge = adge.loc[['Adelie', 'Gentoo']]
adge

species
Adelie    3706.164384
Gentoo    5092.436975
Name: body_mass_g, dtype: float64

In [44]:
# e)
adgebi = pingviner.groupby(['island', 'species'])['body_mass_g'].mean().drop(['Dream', 'Torgersen'], axis=0)
adgebi

island  species
Biscoe  Adelie     3709.659091
        Gentoo     5092.436975
Name: body_mass_g, dtype: float64

In [45]:
# f)
def antallOver(serie, grense=None):
    if grense == None:
        grense = serie.mean()
    return serie[serie > grense].count()


count = pingviner.groupby('species')[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']].agg(
    antallOver)
count

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
species,,,,
Adelie,73,75,66,69
Chinstrap,39,34,35,31
Gentoo,56,62,53,58


In [46]:
# g)
def mm_to_inch(mm):
    return mm / 25


def g_to_k(gram):
    return (gram / 1000)


d = {'species': lambda x: x, 'island': lambda x: x, 'bill_length_mm': mm_to_inch, 'flipper_length_mm': mm_to_inch,
     'body_mass_g': g_to_k, 'sex': lambda x: x}

resultat = pingviner.transform(d)
resultat.columns = ['species', 'island', 'bill_length', 'flipper_length', 'body_mass', 'sex']
resultat

,species,island,bill_length,flipper_length,body_mass,sex
0,Adelie,Torgersen,1.564,7.24,3.750,MALE
1,Adelie,Torgersen,1.580,7.44,3.800,FEMALE
2,Adelie,Torgersen,1.612,7.80,3.250,FEMALE
3,Adelie,Torgersen,1.468,7.72,3.450,FEMALE
4,Adelie,Torgersen,1.572,7.60,3.650,MALE
...,...,...,...,...,...,...
328,Gentoo,Biscoe,1.888,8.56,4.925,FEMALE
329,Gentoo,Biscoe,1.872,8.60,4.850,FEMALE
330,Gentoo,Biscoe,2.016,8.88,5.750,MALE
331,Gentoo,Biscoe,1.808,8.48,5.200,FEMALE


In [47]:
# h)
# 1)
tallisland = pingviner.groupby('island')['species'].count()
for island, antall in tallisland.items():
    print(f'På {island} er det {antall} individer')

På Biscoe er det 163 individer
På Dream er det 123 individer
På Torgersen er det 47 individer


In [48]:
# i)
def test(gruppe):
    return gruppe['island'].count() > 100


gruppering = pingviner.groupby('island')
resultat = gruppering.filter(test)['body_mass_g'].mean()
resultat

4288.986013986014

In [49]:
# Oppg 2
data = [{'a': 1, 'b': 10, 'c': 100}, {'a': 2, 'b': 20, 'c': 200}, {'a': 3, 'b': 30, 'c': 300}]
df2 = pd.DataFrame(data)


# a)
def rammeArg(df, arg, axis=0):
    if axis == 1:
        series = [df.iloc[r] for r in df.index]
    else:
        series = [df[k] for k in df.columns]
    return pd.concat(series).reset_index(drop=True).agg(arg)

In [50]:
print(rammeArg(df2, sum))
print(rammeArg(df2, np.sum))

666
666


In [51]:
# b)
def maxEndring(serie):
    res = 0
    for p in range(0, len(serie) - 1):
        d = serie.iloc[p + 1] - serie.iloc[p]
        if np.abs(d) > np.abs(res): res = d
    return res

In [52]:
print(rammeArg(df2, maxEndring, axis=0))
print(rammeArg(df2, maxEndring, axis=1))

100
270


In [53]:
# Del 2
# Oppg 1
# a)
ansatte = pd.read_csv('ansatte.csv', sep='\t', parse_dates=['Fdato', 'Adato'])
ansatte

,Navn,AnsattNr,Fdato,Kjønn,Lønn,Avd,Adato
0,Erik Dal,1201,1995-01-09,M,500000,5,2019-10-04
1,Kari Normann,1234,1985-12-08,K,600000,5,2021-02-13
2,Berit Berg,2234,1998-01-19,K,450000,4,2020-11-18
3,Ina Fjell,3201,1981-06-21,K,630000,4,2021-12-01
4,Geir Olsen,3251,1992-09-15,M,580000,5,2019-09-21
5,Jon Dale,4411,2002-07-31,M,450000,5,2021-02-13
6,Jens Smith,4422,1999-03-29,M,450000,4,2022-05-14
7,Anne Solberg,4567,1977-11-10,K,750000,1,2018-03-01


In [58]:
time = datetime.now()
idag = datetime(time.year, time.month, time.day)
resultat = pd.concat([ansatte['Navn'], (idag - ansatte['Fdato'])], axis=1)
resultat.columns = ['Navn', 'Alder']
resultat

,Navn,Alder
0,Erik Dal,10500 days
1,Kari Normann,13819 days
2,Berit Berg,9394 days
3,Ina Fjell,15450 days
4,Geir Olsen,11346 days
5,Jon Dale,7740 days
6,Jens Smith,8960 days
7,Anne Solberg,16769 days


In [64]:
# b)
now = datetime.now()
for i in ansatte.index:
    (navn, anr, fdato, kjønn, lønn, avd, adato) = ansatte.iloc[i].to_list()
    print(f'{navn}, Har vært ansatt i, {(now - adato).days} dager')

Erik Dal, Har vært ansatt i, 1466 dager
Kari Normann, Har vært ansatt i, 968 dager
Berit Berg, Har vært ansatt i, 1055 dager
Ina Fjell, Har vært ansatt i, 677 dager
Geir Olsen, Har vært ansatt i, 1479 dager
Jon Dale, Har vært ansatt i, 968 dager
Jens Smith, Har vært ansatt i, 513 dager
Anne Solberg, Har vært ansatt i, 2048 dager
